# Data Preparation

# Import Data

In [2]:
import sys
sys.path.insert(1, r'C:\Users\rafae\Personal\Github\Functions\Data-Science-Functions')
sys.path.insert(2, r'C:\Users\rafae\Personal\Github\Functions\Data-Science-Functions\Functions')

import Visualizations_Functions as viz 
import Classification_CV as class_cv
import DataPreparation as preprocess 
import FeatureExtraction as ft_extract 
import DataScalling as scalling
import FeatureSelection as ft_selection

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings('ignore')

#set preferences
sns.set_style('whitegrid')

def getTarget (df_target, newTarget = False):
    
    if newTarget == False:
        print('Normal CAPM is being used')
        print('Formula = Ri - (Beta*Rm)')
        df_target = df_target.rename(columns = {'futAlpha':'excessReturns'})
        
    if newTarget==True:
        print('Adjusted CAPM is being used')
        print('Formula = (Ri *abs(Beta))- (*Rm)')
        path = r'D:\Desktop\Long-Term-Model\Target\newTarget_CAPM.csv'
        df_target = pd.read_csv(path)
        df_target['date'] = pd.to_datetime(df_target['date'],).dt.tz_localize(None)
        df_target['fillingDate'] = pd.to_datetime(df_target['fillingDate']).dt.tz_localize(None)
        df_target['excessReturns'] = df_target['newAlpha']
    
    thresh_list = [0,0.05,0.15]
    for thresh in thresh_list:
        df_target.loc[df_target['excessReturns']>=thresh,f'Target_{thresh}'] = 1
        df_target[f'Target_{thresh}'] = df_target[f'Target_{thresh}'].fillna(0)
        
    return(df_target)


def filter_tickers (df):
    growth_cols = ['revenue', 'costOfRevenue',
       'grossProfit', 'researchAndDevelopmentExpenses', 'costAndExpenses',  'ebitda', 'operatingIncome',
            'netIncome', 'eps','quarter','year']
    
    df_temp = df[growth_cols].copy()
    df_temp['quarter'] =pd.to_datetime(df_temp.reset_index()['date']).dt.quarter.values
    df_temp['year'] =pd.to_datetime(df_temp.reset_index()['date']).dt.year.values
    
    growth_df = df_temp.reset_index().set_index(['symbol','year','quarter']).drop(columns = ['fillingDate','date']).unstack('symbol')

    ticker_eliminate = growth_df.isnull().sum().groupby('symbol').median().sort_values(ascending=False) / len(growth_df)
    tickers = ticker_eliminate[ticker_eliminate<=0.05].index
    return(tickers)

def firstPreparation (df):
    #create a copy of the original df to clean
    df_clean = df.copy()
    #remove tickers that have more than 5% of the periods missing
    tickers = filter_tickers(df_clean)
    df_clean = df_clean.loc[df_clean.index.get_level_values(0).isin(tickers)]
    
    return(df_clean)

def calculateDiffDays (df):
    
    """""""""""""""
    Calculate nr of days between filing date and predict date 
    
    """""""""""""""
    df['date_diff'] = (pd.to_datetime(df['futDate']) - pd.to_datetime(df['Date_2']))
    days_diff_list = []
    for days_diff in df['date_diff'].astype(str).str.split(' '):
        days_diff_list.append(days_diff[0])
        
        
    df['date_diff'] = days_diff_list
    return(df)
    

def importData (cloud = False, newTarget=False):
    
    #import from cloud
    if cloud==True:
        projectID = 'stockmarket-v0'

        query = """
                SELECT * FROM `stockmarket-v0.stockMarket_dev.quarterlyAlphaDataset` as df
                
                """
                
        df = pd.read_gbq(query= query, 
                        project_id=projectID).set_index(['symbol','date','fillingDate'])
        
    #import using path
    if cloud==False: 
        df = pd.read_csv(path).set_index(['symbol','date','fillingDate']).drop(columns = 'Unnamed: 0')
        
    #filter observations based on date
    df = df.loc[(df.index.get_level_values(1)>='2003-01-01')&
                (df.index.get_level_values(1)<'2023-01-01')]

    #reset index
    df = df.reset_index()
    #set datetime featres
    df['date'] = pd.to_datetime(df['date']).dt.tz_localize(None)
    df['fillingDate'] = pd.to_datetime(df['fillingDate']).dt.tz_localize(None)
    df = df.set_index(['symbol','date','fillingDate'])

    #create year and quarter based in fillingDate
    df['quarter'] = pd.to_datetime(df.index.get_level_values(2)).quarter
    df['year'] = pd.to_datetime(df.index.get_level_values(2)).year
    
    #generate target
    df_target = getTarget(df.iloc[:,-5:],
                          newTarget=newTarget)

    #calculate nr of days between fillingDate and date to predict future returns
    df = calculateDiffDays(df)

    #drop leakage columns
    to_drop = ['futDate',
        'futstockPrice', 'futindexPrice', 'futAlpha']

    df_clean = firstPreparation(df.drop(columns = to_drop))
    
    
    nr_obs = df_clean.shape[0]
    n_cols = len(df_clean.columns)
    print(f'Nr of observations : {nr_obs}')
    print(f'Nr of columns : {n_cols}')
    
    return(df_clean, df_target)


#set color for graphs
color1 = 'royalblue'


input_path = r'E:\Database\Reasearch Topic\Long-Term'
data_path = r'fundamentalData_v2.csv'
target_path = r'CAPM\QuarterlyTarget_CAPM.csv'
resample_path = r'QuarterlyPrices.csv'

path = r'E:\Database\Reasearch Topic\Long-Term\fundamentalData.csv'


df_clean, df_target = importData(cloud=False,
                                 newTarget=False)


Normal CAPM is being used
Formula = Ri - (Beta*Rm)
Nr of observations : 121407
Nr of columns : 176


## Data Cleaning 

__Steps:__  
- Fix column names and eliminate duplicate columns  
- Check data types and correct them

In [4]:
df = df_clean.copy()
print('Column Names:')
for col in df.columns:
    print(col)

Column Names:
reportedCurrency
acceptedDate
calendarYear
period
revenue
costOfRevenue
grossProfit
grossProfitRatio
researchAndDevelopmentExpenses
generalAndAdministrativeExpenses
sellingAndMarketingExpenses
sellingGeneralAndAdministrativeExpenses
otherExpenses
operatingExpenses
costAndExpenses
interestIncome
interestExpense
depreciationAndAmortization
ebitda
ebitdaratio
operatingIncome
operatingIncomeRatio
totalOtherIncomeExpensesNet
incomeBeforeTax
incomeBeforeTaxRatio
incomeTaxExpense
netIncome
netIncomeRatio
eps
epsdiluted
weightedAverageShsOut
weightedAverageShsOutDil
link
finalLink
cashAndCashEquivalents
shortTermInvestments
cashAndShortTermInvestments
netReceivables
inventory
otherCurrentAssets
totalCurrentAssets
propertyPlantEquipmentNet
goodwill
intangibleAssets
goodwillAndIntangibleAssets
longTermInvestments
taxAssets
otherNonCurrentAssets
totalNonCurrentAssets
otherAssets
totalAssets
accountPayables
shortTermDebt
taxPayables
deferredRevenue
otherCurrentLiabilities
totalCurren

__Eliminate duplicate columns:__  
During the pipeline that integrates multiple data sources in a single source, some columns that were repeated across sources were merged together. The goal is to identify those columns, eliminate the one that are duplicate and then rename the name of the others.



In [17]:
#get columns that have _x or _y or _z since this are probably duplicated columns
duplicated_cols = df.columns[(df.columns.str.contains('_x'))| 
                            (df.columns.str.contains('_y'))|
                            (df.columns.str.contains('_z'))]

display(df[duplicated_cols])



Empty DataFrame
Columns: []
Index: [(AAL, 2012-03-31 00:00:00, 2012-04-19 00:00:00), (AAL, 2003-06-30 00:00:00, 2003-06-30 00:00:00), (AAL, 2018-06-30 00:00:00, 2018-07-26 00:00:00), (AAL, 2016-06-30 00:00:00, 2016-07-22 00:00:00), (AAL, 2021-12-31 00:00:00, 2022-02-22 00:00:00), (AAL, 2021-03-31 00:00:00, 2021-04-22 00:00:00), (AAL, 2005-06-30 00:00:00, 2005-06-30 00:00:00), (AAL, 2015-09-30 00:00:00, 2015-10-23 00:00:00), (AAL, 2004-09-30 00:00:00, 2004-09-30 00:00:00), (AAL, 2004-12-31 00:00:00, 2005-02-25 00:00:00), (AAL, 2013-12-31 00:00:00, 2014-02-28 00:00:00), (AAL, 2006-09-30 00:00:00, 2006-10-20 00:00:00), (AAL, 2012-12-31 00:00:00, 2013-02-20 00:00:00), (AAL, 2018-12-31 00:00:00, 2019-02-25 00:00:00), (AAL, 2007-03-31 00:00:00, 2007-04-20 00:00:00), (AAL, 2014-03-31 00:00:00, 2014-04-24 00:00:00), (AAL, 2011-09-30 00:00:00, 2011-10-19 00:00:00), (AAL, 2011-12-31 00:00:00, 2012-02-15 00:00:00), (AAL, 2008-03-31 00:00:00, 2008-03-31 00:00:00), (AAL, 2016-03-31 00:00:00, 2016-04-22 00:00:00), (AAL, 2006-03-31 00:00:00, 2006-04-20 00:00:00), (AAL, 2020-03-31 00:00:00, 2020-04-30 00:00:00), (AAL, 2013-06-30 00:00:00, 2013-07-18 00:00:00), (AAL, 2019-12-31 00:00:00, 2020-02-19 00:00:00), (AAL, 2010-03-31 00:00:00, 2010-04-21 00:00:00), (AAL, 2011-03-31 00:00:00, 2011-04-20 00:00:00), (AAL, 2015-06-30 00:00:00, 2015-07-24 00:00:00), (AAL, 2017-09-30 00:00:00, 2017-10-26 00:00:00), (AAL, 2005-03-31 00:00:00, 2005-03-31 00:00:00), (AAL, 2012-09-30 00:00:00, 2012-10-17 00:00:00), (AAL, 2018-03-31 00:00:00, 2018-04-26 00:00:00), (AAL, 2009-03-31 00:00:00, 2009-03-31 00:00:00), (AAL, 2019-09-30 00:00:00, 2019-10-24 00:00:00), (AAL, 2017-03-31 00:00:00, 2017-04-27 00:00:00), (AAL, 2010-12-31 00:00:00, 2011-02-16 00:00:00), (AAL, 2016-09-30 00:00:00, 2016-10-20 00:00:00), (AAL, 2005-12-31 00:00:00, 2006-02-24 00:00:00), (AAL, 2022-12-31 00:00:00, 2023-02-22 00:00:00), (AAL, 2007-12-31 00:00:00, 2007-12-31 00:00:00), (AAL, 2010-06-30 00:00:00, 2010-07-21 00:00:00), (AAL, 2008-06-30 00:00:00, 2008-06-30 00:00:00), (AAL, 2022-06-30 00:00:00, 2022-07-21 00:00:00), (AAL, 2003-03-31 00:00:00, 2003-03-31 00:00:00), (AAL, 2019-06-30 00:00:00, 2019-07-25 00:00:00), (AAL, 2019-03-31 00:00:00, 2019-04-26 00:00:00), (AAL, 2018-09-30 00:00:00, 2018-10-25 00:00:00), (AAL, 2017-06-30 00:00:00, 2017-07-28 00:00:00), (AAL, 2009-09-30 00:00:00, 2009-09-30 00:00:00), (AAL, 2009-06-30 00:00:00, 2009-06-30 00:00:00), (AAL, 2014-12-31 00:00:00, 2015-02-25 00:00:00), (AAL, 2021-09-30 00:00:00, 2021-10-21 00:00:00), (AAL, 2013-09-30 00:00:00, 2013-10-17 00:00:00), (AAL, 2020-06-30 00:00:00, 2020-07-23 00:00:00), (AAL, 2010-09-30 00:00:00, 2010-10-20 00:00:00), (AAL, 2022-09-30 00:00:00, 2022-10-20 00:00:00), (AAL, 2003-12-31 00:00:00, 2004-02-27 00:00:00), (AAL, 2014-09-30 00:00:00, 2014-10-23 00:00:00), (AAL, 2004-03-31 00:00:00, 2004-04-23 00:00:00), (AAL, 2004-06-30 00:00:00, 2004-06-30 00:00:00), (AAL, 2013-03-31 00:00:00, 2013-04-18 00:00:00), (AAL, 2006-06-30 00:00:00, 2006-07-25 00:00:00), (AAL, 2009-12-31 00:00:00, 2010-02-17 00:00:00), (AAL, 2021-06-30 00:00:00, 2021-07-22 00:00:00), (AAL, 2017-12-31 00:00:00, 2018-02-21 00:00:00), (AAL, 2016-12-31 00:00:00, 2017-02-22 00:00:00), (AAL, 2015-03-31 00:00:00, 2015-04-24 00:00:00), (AAL, 2015-12-31 00:00:00, 2016-02-24 00:00:00), (AAL, 2005-09-30 00:00:00, 2005-09-30 00:00:00), (AAL, 2020-09-30 00:00:00, 2020-10-22 00:00:00), (AAL, 2014-06-30 00:00:00, 2014-07-24 00:00:00), (AAL, 2008-09-30 00:00:00, 2008-09-30 00:00:00), (AAL, 2022-03-31 00:00:00, 2022-04-21 00:00:00), (AAL, 2008-12-31 00:00:00, 2008-12-31 00:00:00), (AAL, 2012-06-30 00:00:00, 2012-07-18 00:00:00), (AAL, 2006-12-31 00:00:00, 2007-02-23 00:00:00), (AAL, 2003-09-30 00:00:00, 2003-09-30 00:00:00), (AAL, 2011-06-30 00:00:00, 2011-07-20 00:00:00), (AAL, 2020-12-31 00:00:00, 2021-02-17 00:00:00), (AAL, 2007-06-30 00:00:00, 2007-07-24 00:00:00), (AAON, 2021-03-31 00:00:00, 2021-05-06 00:00:00), (AAON, 2015-09-30 00:00:00, 2015-11-02 00:0

__Check Data-types:__

In [18]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 89936 entries, ('AAL', Timestamp('2012-03-31 00:00:00'), Timestamp('2012-04-19 00:00:00')) to ('ZUMZ', Timestamp('2022-04-30 00:00:00'), Timestamp('2022-06-06 00:00:00'))
Columns: 176 entries, reportedCurrency to date_diff
dtypes: float64(166), object(10)
memory usage: 121.7+ MB


In [19]:
for col , type_ in zip(df_clean.dtypes.index, df_clean.dtypes):
    print(f'Column {col}: {type_}')

Column reportedCurrency: object
Column acceptedDate: object
Column calendarYear: float64
Column period: object
Column revenue: float64
Column costOfRevenue: float64
Column grossProfit: float64
Column grossProfitRatio: float64
Column researchAndDevelopmentExpenses: float64
Column generalAndAdministrativeExpenses: float64
Column sellingAndMarketingExpenses: float64
Column sellingGeneralAndAdministrativeExpenses: float64
Column otherExpenses: float64
Column operatingExpenses: float64
Column costAndExpenses: float64
Column interestIncome: float64
Column interestExpense: float64
Column depreciationAndAmortization: float64
Column ebitda: float64
Column ebitdaratio: float64
Column operatingIncome: float64
Column operatingIncomeRatio: float64
Column totalOtherIncomeExpensesNet: float64
Column incomeBeforeTax: float64
Column incomeBeforeTaxRatio: float64
Column incomeTaxExpense: float64
Column netIncome: float64
Column netIncomeRatio: float64
Column eps: float64
Column epsdiluted: float64
Colum

In [20]:
nr_obs = df.shape[0]
n_cols = len(df_clean.columns)
print(f'Nr of observations : {nr_obs}')
print(f'Nr of columns : {n_cols}')

Nr of observations : 89936
Nr of columns : 176


# Data Preparation

### Feature Engineer

In [5]:
def genGrowthV2 (df, growth_cols = None):
    
    """""""""""
    Receives:
        pandas df with quarterly data 
        growth_cols - columns where growth will be calculated
    
    """""""""""
    
    #get growth columns
    if growth_cols == None:
        growth_cols = ['revenue', 'costOfRevenue',
       'grossProfit', 'researchAndDevelopmentExpenses', 'costAndExpenses',  'ebitda', 'operatingIncome',
            'netIncome', 'eps','quarter','year']

    #filter data to get only growth columns
    df_temp = df[growth_cols].copy()
    df_temp['quarter'] =pd.to_datetime(df_temp.reset_index()['date']).dt.quarter.values
    df_temp['year'] =pd.to_datetime(df_temp.reset_index()['date']).dt.year.values

    dates_temp = df_temp.reset_index().set_index(['symbol','year','quarter']).unstack('symbol')[['date','fillingDate']]
    date = dates_temp.stack('symbol')

    growth_df = df_temp.reset_index().set_index(['symbol','year','quarter']).drop(columns = ['fillingDate','date']).unstack('symbol')

    #calculate QoQ growth
    QoQ_growth = growth_df.pct_change()
    QoQ_growth.replace([np.inf], 1, inplace=True)
    QoQ_growth.replace([-np.inf], -1, inplace=True)
    QoQ_growth = QoQ_growth.iloc[1:]

    QoQ_growth = QoQ_growth.fillna( QoQ_growth.median())
    QoQ_growth = QoQ_growth.stack('symbol').fillna(0)

    cols = QoQ_growth.columns
    for col in cols:
            QoQ_growth.rename(columns = {col:f'QoQ_{col}'},
                             inplace=True)
            
    #calculate YoY growth
    YoY_growth = growth_df.pct_change(4)
    YoY_growth.replace([np.inf], 0, inplace=True)
    YoY_growth.replace([-np.inf], 0, inplace=True)
    YoY_growth = YoY_growth.iloc[4:]

    YoY_growth = YoY_growth.stack('symbol')
    YoY_growth = YoY_growth.unstack('symbol')
    YoY_growth = YoY_growth.fillna( YoY_growth.median())
    YoY_growth = YoY_growth.stack('symbol').fillna(0)


    for col in cols:
            YoY_growth.rename(columns = {col:f'YoY_{col}'},
                             inplace=True)

    #merge data
    YoY_growth = pd.merge(YoY_growth,
                          date.reset_index().rename(columns = {0:'date'}),
            on = ['symbol','year','quarter'],
            how='left').set_index(['symbol','year','quarter'])


    full_growth = pd.merge(YoY_growth,
            QoQ_growth,
            how = 'left',
            on = ['symbol','quarter','year'])
    
    #remove columns where growth can't be calculated 
    full_growth = full_growth.loc[full_growth['date'].isnull()==False]
    return(full_growth, growth_cols)

def gen_TTMV2 (df, ttm_cols = None):
    
    if ttm_cols == None:
        ttm_cols = ['revenue', 'costOfRevenue','grossProfit',
                'researchAndDevelopmentExpenses', 'costAndExpenses', 'ebitda', 'operatingIncome',
                'netIncome', 'eps']

    df_temp = df[ttm_cols].copy()
    df_temp['quarter'] =pd.to_datetime(df_temp.reset_index()['date']).dt.quarter.values
    df_temp['year'] =pd.to_datetime(df_temp.reset_index()['date']).dt.year.values

    dates_temp = df_temp.reset_index().set_index(['symbol','year','quarter']).unstack('symbol')[['date','fillingDate']]
    date = dates_temp.stack('symbol')

    growth_df = df_temp.reset_index().set_index(['symbol','year','quarter']).drop(columns = ['fillingDate','date']).unstack('symbol')


    ttm_df = growth_df.rolling(4).sum()
    ttm_df = ttm_df.iloc[4:]
    ttm_df = ttm_df.stack('symbol')
    
    cols = ttm_df.columns
    for col in cols:
            ttm_df.rename(columns = {col:f'{col}_TTM'},
                             inplace=True)
            
    ttm_df = pd.merge(ttm_df,
                    date.reset_index().rename(columns = {0:'date'}),
            on = ['symbol','year','quarter'],
            how='left').set_index(['symbol','year','quarter'])
            
    return(ttm_df)


def fund_df_engineer_0 (fund_df):
    #calculate ebit and earnings
    fund_df['ebit'] = fund_df['ebitda'] - fund_df['depreciationAndAmortization']
    fund_df['earnings'] =fund_df['ebit'] -  (fund_df['interestExpense'] + fund_df['incomeTaxExpense'])
    fund_df['totalDividend'] = fund_df['dividend'] * fund_df['weightedAverageShsOut']
    return(fund_df)


def mergeValue (full_eng):
    project_id = 'stockmarket-v0'
    query = """
    SELECT * 
    FROM `stockmarket-v0.stockMarket_dev.enterpriseValue` as df

    """
    df_value = pd.read_gbq(query = query, project_id = project_id)
    df_value['date'] = pd.to_datetime(df_value['date'])
    full_eng = pd.merge(full_eng.reset_index(),
                        df_value,
                        on = ['symbol','date'],
                        how='left')
    return(full_eng)


def importIndicators (cloud = False):
    #import indicators from bigquery
    if cloud ==True:
        print('Import technical indicators from cloud')
        query = """
        SELECT *
        FROM `stockmarket-v0.stockMarket_dev.QuarterlyTechnicalIndicators`as df
        """
        projectID = 'stockmarket-v0'
        df_indicators = pd.read_gbq(query = query, project_id= projectID)
    #import indicators from csv (local machine) 
    if cloud ==False:
        print('Import technical indicators from local disk')
        path = r'D:\Desktop\Long-Term-Model\Data\data_Technicalindicators.csv'
        df_indicators = pd.read_csv(path).drop(columns = 'Unnamed: 0')
    
    df_indicators = df_indicators.loc[df_indicators.MA_21.isnull()==False]
    df_indicators['fillingDate'] = pd.to_datetime(df_indicators['fillingDate'])
    
    return(df_indicators)


def eliminateGaps (data):
    data['lastFilling'] = data.reset_index().groupby('symbol').shift(1)['fillingDate'].values
    data['fillingDiff'] = pd.to_datetime(data['fillingDate']) - pd.to_datetime(data['lastFilling'])
    
    data = data.loc[data['fillingDiff']<'200 days']
    return(data)

def replace_inf_with_zero(df):
    # Replace infinite values with 0
    df.replace([np.inf, -np.inf], 0, inplace=True)
    return df

def full_FT_Eng (df_clean):
    df_eng = df_clean.copy()
    
    #create columns related to financial ratios
    df_eng = fund_df_engineer_0(df_eng)

    ttm_cols = ['revenue', 'costOfRevenue','grossProfit',
                'researchAndDevelopmentExpenses', 'costAndExpenses', 'ebitda', 'operatingIncome',
                'netIncome', 'eps','earnings']

    #calculate TTM and growth columns
    df_ttm = gen_TTMV2(df_eng,ttm_cols)
    df_growth, growth_cols = genGrowthV2(df_eng,)

    #create a copy of DF
    df_eng_ = df_eng

    #sort by symbol and filling date
    df_eng_2 = df_eng_.sort_values(['symbol','fillingDate'])


    #match indexes of the created datasets with the main df
    df_ttm = df_ttm.reset_index().set_index(['symbol','fillingDate','date'])
    df_growth = df_growth.reset_index().set_index(['symbol','fillingDate','date'])
    df_eng_2 = df_eng_2.reset_index().set_index(['symbol','fillingDate','date'])
    
    #filter index of main data to account for lagging features
    df_eng_2 = df_eng_2.loc[df_eng_2.index.isin(df_ttm.index)]


    #merge main data with ttm 
    full_eng = pd.merge(df_eng_2, df_ttm.drop(columns = ['year','quarter']),
                        on = ['symbol','date','fillingDate'],
                        how = 'right')
    
    #merge main data with growth 
    full_eng = pd.merge(full_eng, df_growth.drop(columns = ['year','quarter']),
                        on = ['symbol','date','fillingDate'],
                        how = 'left')
    
    full_eng = mergeValue(full_eng).set_index(['symbol','fillingDate','date'])
    
    
    #imort technical indicators
    df_indicators = importIndicators(cloud=False)
    if 'date' in df_indicators.columns:
        df_indicators = df_indicators.drop(columns = 'date')
    #merge fundamental with technical indicators 
    full_eng = pd.merge(full_eng.reset_index(),
                        df_indicators,
                        on=['symbol','fillingDate'],
                        how='left').set_index(['symbol','date','fillingDate','year','quarter'])
    
    full_eng = replace_inf_with_zero(full_eng)
    
    
    
    nr_obs = full_eng.shape[0]
    n_cols = len(full_eng.columns)
    print(f'Nr of observations : {nr_obs}')
    print(f'Nr of columns : {n_cols}')
    
    return(full_eng)

full_eng = full_FT_Eng(df_clean)


Import technical indicators from local disk
Nr of observations : 115699
Nr of columns : 223


#### Drop observations with missmatched Financial Ratios

In [6]:
#drop tickers where financial ratios are missmatched
tickers_toDrop = full_eng.loc[full_eng['priceToSalesRatio'].isnull()==True].reset_index()['symbol'].unique()
full_eng = full_eng.loc[full_eng.index.get_level_values(0).isin(tickers_toDrop)==False]

nr_obs = full_eng.shape[0]
n_cols = len(full_eng.columns)
print(f'Nr of observations : {nr_obs}')
print(f'Nr of columns : {n_cols}')

Nr of observations : 86516
Nr of columns : 223


## Missing Values

The first step of this stage of the process was to fill missing values in columns related to dividends with 0 since its reasonable to assume that some companies don't pay dividends, after filling those values, remaining columns that have more than 30% missing data were removed. These features were removed because imputing a certain value would introduce a lot of bias that could negatively impact the model.v Only one columns fell into the requirements, namely 'YoY_researchAndDevelopmentExpenses' that had 82% of the data missing.

In [7]:
def remove_cols_missing (x_missing, thresh):
    '''''''''''''''
    receives:
        DF
        thresh - % threshold
    Removes all columns that have more than a certain % missing
        
    '''''''''''''''

    mv_ = (x_missing.isnull().sum() / len(x_missing)).sort_values(ascending=False)
    deleted_ = []
    for col, value in zip(mv_.index, mv_.values):
        if value>=thresh:
            x_missing = x_missing.drop(columns=f'{col}')
            deleted_.append(col)

    print(f'{len(deleted_)} features were removed beucause they has a more than {thresh*100}% of observations missing.')
    print(f'Features removed: {deleted_}')
    return(x_missing)

def interactiveImputer (data, estimator):
    from sklearn.experimental import enable_iterative_imputer  
    from sklearn.impute import  SimpleImputer
    from sklearn.impute import IterativeImputer
    metric_features = data.select_dtypes(include=np.number).columns
    cat_features = data.select_dtypes(exclude=np.number).columns
    
    if len(data)<1000:
        sample_size = len(data)
    else:
        sample_size = 1000
    
    imputer = IterativeImputer(estimator=estimator)
    imputer.fit(data.sample(sample_size)[metric_features])
    data_imputed = imputer.transform(data[metric_features])
    data_imputed = pd.DataFrame(data_imputed, columns=metric_features, index = data.index)
    return(pd.concat([data_imputed,data[cat_features]],axis=1), imputer)

def missingValues (df_missing):
    #get a copy of the df
    #fill dividend missing values with 0 
    dividend_cols = df_missing.columns[(df_missing.columns.str.contains('dividend'))  \
                                       | (df_missing.columns.str.contains('Dividend'))]
    
    #for each column in dividends, fill missing values with 0
    for col in dividend_cols:
        df_missing[col] = df_missing[col].fillna(0)

    #for every columns more than  30% of the data missing
    thresh = 0.30
    df_missing = remove_cols_missing(df_missing.copy(),thresh)
    
    #get numerical, categoricsal and boolean columns
    num_cols = df_missing.select_dtypes(include = np.number).columns
    cat_cols = df_missing.select_dtypes(exclude = np.number).columns
    bool_cols = df_missing.select_dtypes(include = np.bool).columns
    
    return(df_missing, num_cols,cat_cols,bool_cols)

def imputMissingValues (df):
    print('Features with most missing values:')
    print(full_eng.isnull().sum().sort_values(ascending = False).iloc[:15])
    print(' ')
    print('Features with most missing values, in %:')
    print(' ')
    print((full_eng.isnull().sum()/len(full_eng)).sort_values(ascending = False).iloc[:15])
    df_missing, num_cols,cat_cols,bool_cols = missingValues(df)
    
    
    from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
    num_cols = df_missing.select_dtypes(include = np.number).columns
    cat_cols = df_missing.select_dtypes(include = np.bool_).columns
    bool_cols = df_missing.select_dtypes(include = np.object).columns

    model = DecisionTreeRegressor(random_state=0,
                                  max_depth=3)
    
    print('Imputing missing data with interactive imputer- Decision Tree is base model')
    #imput missing values in numerical features using interactive imputer with decision tree
    df_imput_dt, imputer = interactiveImputer(df_missing[num_cols],
                                                 model)

    #concat df with num features with df with bool and categorical features
    df_imput = pd.concat([df_imput_dt, 
                          df_missing[bool_cols]],
                         axis=1)
    df_imput = pd.concat([df_imput,
                          df_missing[cat_cols]],
                         axis=1)
    
#     fill missing values in sector and industry with unkown
    df_imput.loc[df_imput['sector'].isnull()==True,
                 'sector'] = 'Unkown'
    df_imput.loc[df_imput['industry'].isnull()==True,
                 'industry'] = 'Unkown'
    
    mv_cat = df_imput.isnull().sum().sort_values(ascending=False).iloc[:10]
    mv_cat = mv_cat[mv_cat.values>0].index

    mv_ = df_imput.select_dtypes(include = np.number).isnull().sum().sum()
    print(f'Nr of missing values in numerical features after interactive imputer: {mv_}')
    mv_ = df_imput.select_dtypes(exclude = np.number).isnull().sum().sum()
    print(f'Nr of missing values in categorical features after interactive imputer: {mv_}')

    print(f'Features with missing values {list(mv_cat)}')
    
    return(df_imput)

df_imput = imputMissingValues(full_eng)

Features with most missing values:
dividend               40940
totalDividend          40940
floatShares            34848
operatingCycle         26999
cashConversionCycle    26999
MA_42                  11544
MA_21                  11544
Volatility_21          11544
excessReturn_21        11544
excessReturn_84        11544
Volatility_42          11544
MA_63                  11544
excessReturn_42        11544
Volatility_84          11544
Volatility_63          11544
dtype: int64
 
Features with most missing values, in %:
 
dividend               0.473207
totalDividend          0.473207
floatShares            0.402793
operatingCycle         0.312069
cashConversionCycle    0.312069
MA_42                  0.133432
MA_21                  0.133432
Volatility_21          0.133432
excessReturn_21        0.133432
excessReturn_84        0.133432
Volatility_42          0.133432
MA_63                  0.133432
excessReturn_42        0.133432
Volatility_84          0.133432
Volatility_63          0

## Feature Engineer - After Preparation

In [8]:
def gen_DFModel (df_imput,df_target):
    #drop duplicates and set new index
    df_full = df_imput.reset_index().drop_duplicates(['symbol','date']).set_index(['symbol','date','fillingDate'])

    #drop duplicates and set new index
    df_target = df_target.reset_index().drop_duplicates(['symbol','date']).set_index(['symbol','date','fillingDate'])

    df_model = pd.merge(df_full,
                        df_target.iloc[:,:],
                        on = ['symbol','date','fillingDate'],
                        how='left')

    #drop observations where the target is null 
    #might be companies where daily price isn't available 
    #might be targets can't be calculated 
    df_model = df_model.loc[(df_model['Target_0.05'].isnull()==False)&
                           (df_model['excessReturns'].isnull()==False)]
    
    df_model['date_diff'] = df_model['date_diff'].astype(int)
    
    return(df_model)

#incoherence checking
def perform_Incoherence (data):
    """""""""
    Receives:
        data - pandas df after being merged with Target
        
    Removes incoherences from the dataset 
    
    """""""""
    df_inc = data.copy()
    #set incoherences
    df_inc = df_inc.loc[df_inc['totalDebt']>=0]
    df_inc = df_inc.loc[df_inc['totalAssets']>0]
    df_inc = df_inc.loc[df_inc['revenue']>=0]
    df_inc = df_inc.loc[df_inc['weightedAverageShsOut']>=0]
    df_inc = df_inc.loc[df_inc['date_diff']<100]

    df_inc_ = data.loc[data.index.isin(df_inc.index)]
    init_len = len(data)
    pct_rem = np.round(len(df_inc_) / init_len, 3)

    print(f'{pct_rem*100}% of the data remained after pre-processing')
    
    return(df_inc_)


def get_Binaries(df):
    
    col_name = 'priceEarningsRatio'
    new_col = 'sector_priceEarningsRatio_bin'
    for sector in df['sector'].unique():
        for quarter in df.reset_index()['quarter'].unique():
            for year in df.reset_index()['year'].unique():
                cond_industry  = (df['sector']==f'{sector}')
                cond_quarter = (df.index.get_level_values(4)==quarter)
                cond_year = (df.index.get_level_values(3)==year)
                val = df.loc[ cond_quarter & cond_year, col_name].quantile(0.2)
                cond_value =  (df[col_name]<val)
                df.loc[ cond_quarter & cond_year & \
                       cond_value   ,new_col] = 1
    df[new_col] = df[new_col].fillna(0).astype(bool)
    
    
    col_name = 'netDebt'
    new_col = 'netDebt_bin'
    division = 'sector'
    for sector in df[division].unique():
        for quarter in df.reset_index()['quarter'].unique():
            for year in df.reset_index()['year'].unique():
                cond_industry  = (df[division]==f'{sector}')
                cond_quarter = (df.index.get_level_values(4)==quarter)
                cond_year = (df.index.get_level_values(3)==year)
                val = df.loc[cond_quarter & cond_year, col_name].quantile(0.2)
                cond_value =  (df[col_name]<val)
                df.loc[  cond_quarter & cond_year & \
                       cond_value   ,new_col] = 1
    df[new_col] = df[new_col].fillna(0).astype(bool)
    
    
    col_name = 'netIncome_TTM'
    new_col = 'netIncome_bin'
    division = 'sector'
    for sector in df[division].unique():
        for quarter in df.reset_index()['quarter'].unique():
            for year in df.reset_index()['year'].unique():
                cond_industry  = (df[division]==f'{sector}')
                cond_quarter = (df.index.get_level_values(4)==quarter)
                cond_year = (df.index.get_level_values(3)==year)
                val = df.loc[cond_quarter & cond_year, col_name].quantile(0.7)
                cond_value =  (df[col_name]<val)
                df.loc[  cond_quarter & cond_year & \
                       cond_value   ,new_col] = 1
    df[new_col] = df[new_col].fillna(0).astype(bool)
    
    col_name = 'netProfitMargin'
    new_col = 'netProfitMargin_bin'
    division = 'sector'
    for sector in df[division].unique():
        for quarter in df.reset_index()['quarter'].unique():
            for year in df.reset_index()['year'].unique():
                cond_industry  = (df[division]==f'{sector}')
                cond_quarter = (df.index.get_level_values(4)==quarter)
                cond_year = (df.index.get_level_values(3)==year)
                val = df.loc[cond_quarter & cond_year, col_name].quantile(0.2)
                cond_value =  (df[col_name]<val)
                df.loc[  cond_quarter & cond_year & \
                       cond_value   ,new_col] = 1



    df[new_col] = df[new_col].fillna(0).astype(bool)
    sectors = ['Services', 'Technology', 'Basic Materials', 'Consumer Cyclical']
    
    df.loc[df['sector'].isin(sectors),'mainSector'] = 1
    df['mainSector'] = df['mainSector'].fillna(0).astype(bool)
    return(df)

def gen_categorical (df):
    '''''''''''
    
    receives 
        pandas df
    
    Generates Marketcap categories based on it's values
    
    '''''''''
    df['mktcap'] = (df['close'] * df['weightedAverageShsOut'])
    
    #create marketcap scales
    bil = (1000000000)

    #if mktcap > 50 billions then mega
    df.loc[df['mktcap']>=bil*50,'marketcap'] = 'Mega'

    #if mktcap  10 < mktcap <50 billion then large
    df.loc[(df['mktcap']>=bil*10) & (df['mktcap']<bil*50),'marketcap'] = 'Large'

    #if mktcap  2 < mktcap <10 billion then mid
    df.loc[(df['mktcap']<bil*10) & (df['mktcap']>=bil*2),'marketcap'] = 'Mid'

    #if mktcap   mktcap <2 billion then small
    df.loc[(df['mktcap']<bil*2) ,'marketcap'] = 'Small'

    #if mktcap   mktcap <300 millions then mucro
    df.loc[(df['mktcap']<300) ,'marketcap'] = 'Micro'

    mkcap_list = ['Small','Mid']
    df.loc[df['marketcap'].isin(mkcap_list),'marketcap_bin'] = 1
    df['marketcap_bin'] = df['marketcap_bin'].fillna(0)
    
    return(df)


def oneHotEncoder (data):
    from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
    non_metric_features =list(data.select_dtypes(exclude=np.number).set_index(data.index).columns)
    ohc = OneHotEncoder(sparse=False)
    ohc_feat = ohc.fit_transform(data[non_metric_features])

    names = ohc.get_feature_names_out()
    
    ohc_cat = pd.DataFrame(data =ohc_feat ,columns = names, index = data.index)
    return(ohc_cat)


def analyzeQuantiles (data,):
    
    """"""""""""""""
    
    Receives:
            pandas dataframe
            
    Creates a df that analyzes outliers by creating 2 disparity features that measure the level of the outliers.
    The fist measure divides de 95% quantile by the 90% and the second divides the mean by the median
    
    Note: thresholds can be further adjusted
    
    """""""""""""""""
    df_out = data
    
    quantile_disp = np.round(df_out.describe([.25, .5, .75,0.9,0.95]).T).iloc[:,:-1]
    quantile_disp['disparity'] = (quantile_disp['95%'] / quantile_disp['90%']).replace(np.inf,0)
    quantile_disp['mean_disp'] = (quantile_disp['mean'] / quantile_disp['50%']).replace(np.inf,0)

    quantile_disp = quantile_disp.sort_values(ascending=False,by = 'mean_disp')
    display(quantile_disp.iloc[:10])
    
    return(quantile_disp)


#replace outliers with quantie 
def replaceOutliers (data,upper_thresh, lower_thresh):
    """"""""""""""""
    
    Receives:
            pandas dataframe
            
    Replaces outliers with the nearest quantile, per example an observation of quantile 99 will be replaced by the quantile 90
    
    Note: thresholds can be fyrther adjusted
    
    """""""""""""""""
    df_out = data.copy()
    
    cols = df_out.select_dtypes(include = np.number).columns
    rem_cols = df_out.select_dtypes(exclude = np.number).columns
    for col in cols:
        
        upper_quant = df_out[col].quantile(upper_thresh)
        lower_quant = df_out[col].quantile(lower_thresh)
        
        df_out.loc[df_out[col]>upper_quant, col] = upper_quant
        df_out.loc[df_out[col]<lower_quant, col] = lower_quant
        
        
    df_out = pd.concat([df_out[cols],
                        df_out[rem_cols]],
                      axis=1)
        
    return(df_out)

#apply log an cubic root transformation
def skewFix (df):
    import numpy as np
    from scipy.stats import skew
    
    def replace_inf_with_zero(df):
        
    # Replace infinite values with 0
        df.replace([np.inf, -np.inf], 0, inplace=True)
        return df
    
    df_transform = df.copy()
    
    for col in df_transform.columns:
        if df_transform[col].dtype ==np.number:
            sk  = skew(df_transform[col])
            min_ = df_transform[col].min()
            if (sk > 1.5) & (min_ >0):
                df_transform[col] = np.log(df_transform[col])

            if (sk > 1.5) & (min_ <=0):
                df_transform[col] = np.cbrt(df_transform[col])

    df_transform = replace_inf_with_zero(df_transform)
    return(df_transform)

def skewTest (df):
    import numpy as np
    from scipy.stats import skew
    
    sk = skew(df)
    
    df_skew = pd.DataFrame(data = sk,
                           index = df.columns,)
    return(df_skew)


def standSector (data):
    quarter_list = data.reset_index()['date'].dt.quarter.unique()
    year_list = data.reset_index()['date'].dt.year.unique()
    sector_list = data.reset_index()['sector'].unique()
    data_norm = pd.DataFrame()
    for year in year_list:
        for quarter in quarter_list:
            for sector in sector_list:
                query_year = data.index.get_level_values(1).year == year
                query_quarter = data.index.get_level_values(1).quarter == quarter
                query_sector = data.sector == sector
                df_temp = data.loc[(query_year) & (query_quarter) & (query_sector)].copy()
                if len(df_temp)>0:
                    df_quantile = ft_extract.quantile_transform_df(df_temp,n_quantiles=100, random_state=0)

                    data_norm = pd.concat([data_norm,df_quantile],
                                          axis=0)
    return(data_norm)

def standQuarter (data):
    quarter_list = data.reset_index()['date'].dt.quarter.unique()
    year_list = data.reset_index()['date'].dt.year.unique()
    
    data_norm = pd.DataFrame()
    for year in year_list:
        for quarter in quarter_list:
            query_year = data.index.get_level_values(1).year == year
            query_quarter = data.index.get_level_values(1).quarter == quarter
            df_temp = data.loc[(query_year) & (query_quarter) ].copy()
            
            if len(df_temp)>0:
                df_quantile = ft_extract.quantile_transform_df(df_temp,n_quantiles=100, random_state=0)

                data_norm = pd.concat([data_norm,df_quantile],
                                      axis=0)
    return(data_norm)


def calcRatios (data):
    #calculate current ratio
    
    #liquidity Ratios
    data['currentRatio'] = data['totalCurrentAssets'] / data['totalCurrentLiabilities']
    data['currentRatio'] = data['currentRatio'].fillna(0)
    
    data['quickRatio'] = (data['cashAndCashEquivalents'] + data['shortTermInvestments'] + data['accountsReceivables'] )/ data['totalCurrentLiabilities']
    data['quickRatio'] = data['quickRatio'].fillna(0)
    
    data['cashRatio'] =  data['cashAndCashEquivalents'] / data['totalCurrentLiabilities'] 
    data['cashRatio'] = data['cashRatio'].fillna(0)
    
    #debt Ratios
    data['debtRatio'] =  data['totalLiabilities'] / data['totalAssets'] 
    data['debtRatio'] = data['debtRatio'].fillna(0)
    
    data['debtEquityRatio']= data['totalDebt']/ data['totalEquity']
    data['debtEquityRatio'] = data['debtEquityRatio'].fillna(0)
    
    #profiitability
    data['grossProfitMargin'] = data['grossProfit'] / data['revenue']
    data['debtEquityRatio'] = data['debtEquityRatio'].fillna(0)
    
    data['operatingProfitMargin'] = data['operatingIncome'] / data['revenue']
    data['operatingProfitMargin'] = data['operatingProfitMargin'].fillna(0)
    
    data['pretaxProfitMargin'] = data['incomeBeforeTax'] /  data['revenue'] 
    data['pretaxProfitMargin'] = data['pretaxProfitMargin'].fillna(0)
    
    data['returnOnAssets'] = data['netIncome'] / data['totalAssets']
    data['returnOnAssets'] = data['returnOnAssets'].fillna(0)
    
    data['returnOnAssets'] = data['netIncome'] / data['totalAssets']
    data['returnOnAssets'] = data['returnOnAssets'].fillna(0)
    
    data['revenuePerShare'] = data['revenue'] / data['numberOfShares']
    data['revenuePerShare'] = data['revenuePerShare'].fillna(0)
    data = replace_inf_with_zero(data)
    return(data)

if 'alpha' in df_target.columns:
    df_target = df_target.drop(columns= 'alpha')
    
if 'beta' in df_target.columns:
    df_target = df_target.drop(columns= 'beta')
    
df_model = gen_DFModel(df_imput,
                       df_target)


    
#set major columns
target_cols = df_model.columns[df_model.columns.str.contains('Target')]

leakage_cols = [ 'futdate','close',
       'futClose', 'futdate', 's&p', 'beta', 'fut_s&p', 'fut_s&p_date',
       'fut_beta', 'futReturns', 'futMktReturns','excessReturns', 'Target_0',
       'Target_0.05', 'Target_0.15']

growth_cols = ['YoY_costOfRevenue', 'YoY_grossProfit', 'YoY_costAndExpenses',
       'YoY_ebitda', 'YoY_operatingIncome', 'YoY_netIncome', 'YoY_eps']

#set columns
ratio_cols = df_model.columns[(df_model.columns.str.contains('Ratio'))]
shares_cols = df_model.columns[(df_model.columns.str.contains('Share'))]
margin_cols = df_model.columns[(df_model.columns.str.contains('Margin'))]
turnover_cols = df_model.columns[(df_model.columns.str.contains('Turnover'))]
dividend_cols = df_model.columns[(df_model.columns.str.contains('dividend'))]


all_colls = [*ratio_cols, *shares_cols, *margin_cols,*turnover_cols,*dividend_cols]
all_colls = set(all_colls)

# df_sector = pd.read_csv(f'{input_path}\companyInfo_sector.csv').drop(columns = 'Unnamed: 0')
print('Nr of rows:')
print(df_model.shape[0])

#perform incoherence checkinng 
df_model = perform_Incoherence(df_model)


##FEATURE ENGINEER 2
#generate categorical features
df_model = gen_categorical(df_model).reset_index().set_index(['symbol','date',
                                                              'fillingDate','year',
                                                              'quarter'])


#create marketcap and marketcap categoricalb 
df_model = gen_categorical(df_model)


 #apply one hot encoder n sector and industry
df_enc = oneHotEncoder(df_model[['sector','industry']])

#concat with main data
df_model = pd.concat([df_model,
                    df_enc],
                    axis=1)

#gen binary features
df_model = get_Binaries(df_model)
#remove excess returns higher than 150%
df_model = df_model.loc[df_model['excessReturns']<1.5]

#define bin columns
bin_cols =['Target_0',
       'Target_0.05', 'Target_0.15','sector_priceEarningsRatio_bin', 'netDebt_bin',
       'netIncome_bin', 'netProfitMargin_bin', 'mainSector']


df_model[bin_cols] = df_model[bin_cols].astype(bool)

bin_cols_ =['sector_priceEarningsRatio_bin', 'netDebt_bin',
       'netIncome_bin', 'netProfitMargin_bin', 'mainSector']


if 'index' in df_model.columns:
    df_model = df_model.drop(columns = 'index')

df_model = calcRatios(df_model)


nr_obs = df_model.shape[0]
n_cols = len(df_model.columns)
print(f'Nr of observations : {nr_obs}')
print(f'Nr of columns : {n_cols}')

#create df skew
df_model_skew = skewFix(df_model)


Nr of rows:
73551
99.0% of the data remained after pre-processing
Nr of observations : 72712
Nr of columns : 382


### Drop Duplicates

In [10]:
def dropDuplicates(df):
    init_len = len(df)
    df = df.reset_index().drop_duplicates(['symbol','fillingDate']).set_index(['symbol','date','fillingDate','year','quarter'])
    
    fin_len = len(df)
    
    print(f'{fin_len/init_len} of the data remained ')
    return(df)


df_model = dropDuplicates(df_model)
df_model_skew = dropDuplicates(df_model_skew)

nr_obs = df_model.shape[0]
n_cols = len(df_model.columns)
print(f'Nr of observations : {nr_obs}')
print(f'Nr of columns : {n_cols}')


0.9976069974694686 of the data remained 
0.9976069974694686 of the data remained 
Nr of observations : 72538
Nr of columns : 382


In [37]:

df_full_path = r'D:\Desktop\Long-Term-Model\Data\LongTerm-DataPreparation_v2.csv'

data = pd.read_csv(df_full_path)
data['date'] = pd.to_datetime(data['date'])
data['fillingDate'] = pd.to_datetime(data['fillingDate'])
data = data.set_index(['symbol','date','fillingDate','year','quarter'])

#### Normalize Data considering Quarter and Sector

In [12]:
leakage_cols = ['alpha', 'futDate', 'excessReturns',
       'Target_0', 'Target_0.05', 'Target_0.15']

data_sector = standSector(df_model.drop(columns=leakage_cols))
data_quarter = standQuarter(df_model.drop(columns=leakage_cols))

### Export Data

- excessReturns corresponds to the Target calculated using the adjusted CAPM formula  
- futAlpha corresponds to the Target calculated using the original CAPM

#### v1

In [13]:
#store on computer memory
df_full_path = r'D:\Desktop\Long-Term-Model\Data\LongTerm-DataPreparation.csv'
df_full_path_skew = r'D:\Desktop\Long-Term-Model\Data\LongTerm-DataPreparation_Skew.csv'
df_model.to_csv(df_full_path)
df_model_skew.to_csv(df_full_path_skew)

#store on computer memory
path_sectorStand = r'D:\Desktop\Long-Term-Model\Data\LongTerm-DataPreparation_sectorStand.csv'
path_quarterStand = r'D:\Desktop\Long-Term-Model\Data\LongTerm-DataPreparation_quarterStand.csv'

data_sector.to_csv(path_sectorStand)
data_quarter.to_csv(path_quarterStand)